# Space Titanic - Bedu Equipo 21
#### *Proyecto de evaluación módulo 4*

In [ ]:
import numpy as np  # NumPy: Biblioteca para cálculos numéricos y manejo de arrays
import pandas as pd  # Pandas: Manipulación y análisis de datos estructurados
import seaborn as sns  # Seaborn: Visualización de datos avanzada basada en Matplotlib
import matplotlib.pyplot as plt  # Matplotlib: Creación de gráficos y visualizaciones

import tensorflow as tf  # TensorFlow: Framework para aprendizaje automático y redes neuronales
from tensorflow.keras.layers import Normalization, Dense, InputLayer  # Keras Layers: Componentes para construir redes neuronales
from tensorflow.keras.losses import MeanAbsoluteError  # Keras Losses: Funciones de pérdida para entrenamiento de modelos
from tensorflow.keras.optimizers import Adam  # Keras Optimizers: Algoritmos de optimización para entrenar modelos
from tensorflow.keras.metrics import RootMeanSquaredError  # Keras Metrics: Métricas para evaluar modelos de aprendizaje automático
from tensorflow.keras.regularizers import l1_l2  # Keras Regularizers: Regularización para prevenir el sobreajuste en modelos
from tensorflow.keras.models import Sequential  # Keras Models: Modelo secuencial para construir redes neuronales de forma lineal
from tensorflow.keras.callbacks import EarlyStopping  # Keras Callbacks: Detención anticipada para evitar sobreajuste durante el entrenamiento
from keras.layers import Dropout  # Keras Layers: Dropout para reducir el sobreajuste en redes neuronales

from sklearn.metrics import roc_curve, auc  # Scikit-learn Metrics: Curva ROC y AUC para evaluación de modelos de clasificación
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay  # Scikit-learn Metrics: Matriz de confusión para evaluación de modelos
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay  # Scikit-learn Metrics: Curva de precisión-recuperación

from scipy.stats import norm  # SciPy Stats: Funciones y distribuciones estadísticas, aquí distribución normal

from sklearn.model_selection import KFold

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Preparación de los datos de Entrenamiento

In [ ]:
# Cargando el dataset de entrenamiento y prueba

df_train_raw = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
df_test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [ ]:
# Mostrando la extensión de cada dataset

print(len(df_train_raw))
print(len(df_test))

In [ ]:
# Primeras 10 filas del set de entrenamiento

df_train_raw.head(10)

In [ ]:
# Tipos de datos
data_type = df_train_raw.dtypes
unique_values = df_train_raw.nunique()
print(data_type, "\n")

print(unique_values)

In [ ]:
# Visualización de la cantidad de valores nulos que se tienen en el dataset de entrenamiento

df_train_raw.isnull().sum()

### Columnas sin relevancia

En el análisis realizado, se identificaron varias columnas como irrelevantes para el estudio. Esta irrelevancia se debe principalmente a dos razones: la presencia de una gran cantidad de datos únicos, tanto numéricos como no numéricos, que están asociados a servicios específicos; y la existencia de un alto número de datos nulos. Las columnas que caen en esta categoría son:

- `Cabin`
- `VIP`
- `RoomService`
- `Foodcourt`
- `ShoppingMall`
- `Spa`
- `VRdeck`
- `Name`

No obstante, es importante destacar una excepción significativa: la columna `Cabin`. Dado el contexto del problema, donde la nave espacial atraviesa una anomalía espaciotemporal, el análisis de esta columna podría ofrecer insights valiosos. Las anomalías espaciotemporales son fenómenos poco comprendidos, lo que nos lleva a plantear hipótesis y cuestionamientos sobre su funcionamiento en relación con los pasajeros de la nave. Algunas preguntas relevantes podrían ser:

- ¿Influye la ubicación de los pasajeros en la nave (indicada por su cabina) en la probabilidad de ser transportados a través de la anomalía?
- ¿Hay una correlación entre la edad de los pasajeros y la tendencia a ser transportados?
- ¿El monto de dinero gastado por los viajeros en servicios de la nave tiene alguna relevancia en su transporte a través de la anomalía?

Gracias a estas preguntas, se hace relevante considerar la ubicación de los pasajeros en la nave, específicamente según la cabina en la que se encuentran. Un dato importante a tener en cuenta es que las cabinas se dividen en dos grupos principales:

- Aquellas ubicadas en la sección **"P"** (Port), que corresponde al lado izquierdo de la nave.
- Aquellas en la sección **"S"** (Starboard), que se refiere al lado derecho de la nave.

Esta clasificación de cabinas en "Port" y "Starboard" podría ser crucial para entender cómo la posición en la nave afecta la experiencia de los pasajeros durante la anomalía espaciotemporal. Podría existir una correlación entre la ubicación de la cabina y la probabilidad de ser afectado por la anomalía, lo cual abre nuevas vías de análisis en nuestro estudio.

La estructura de la columna `Cabin` tiene el formato `deck/num/side`, pero para simplificar el análisis y evitar el manejo de 6560 datos distintos, hemos decidido dividir esta información en dos categorías únicas: **Port (P)** y **Starboard (S)**. Esta división se complementa con la información sobre el sueño criogénico de los pasajeros, permitiéndonos inferir si se encontraban dentro de su cabina en el momento de la transportación. Esto nos ayuda a analizar la probabilidad de que fueran transportados desde una determinada cabina en un lado específico de la nave mientras estaban en criosueño.

Las demás columnas, como `RoomService`, `Foodcourt`, `ShoppingMall`, `Spa`, `VRdeck`, y `Name`, serán eliminadas de nuestro análisis. Estas no muestran una correlación aparente con el fenómeno de transportación que estamos estudiando. 

Más adelante, realizaremos una matriz de correlación para validar nuestras hipótesis acerca de la relación entre la ubicación de la cabina, el estado de criosueño de los pasajeros, y otras variables relevantes en nuestro estudio.


In [ ]:
# Eliminar columnas 

df_train_raw = df_train_raw.drop(columns=["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Name", "VIP"])

# Mostramos el nuevo dataset sin las columnas

df_train_raw.columns


### Análisis por columna

En esta sección, realizaremos una revisión detallada de cada columna para determinar el tratamiento de los datos nulos. Dependiendo de la naturaleza y la importancia de cada columna, decidiremos si eliminamos las filas con datos faltantes o si los rellenamos con un valor estadísticamente significativo, como la media o la moda.

#### Columna `Age`

La columna `Age` representa la edad de los pasajeros. El primer paso es analizar la distribución de los datos de edad para entender su rango, media, mediana y posibles outliers. Esto nos ayudará a decidir si es más adecuado rellenar los datos faltantes con la media o la mediana, o si se requiere de alguna otra técnica de imputación.

1. **Análisis Exploratorio**: Iniciaremos con un análisis exploratorio de la columna `Age`, utilizando estadísticas descriptivas y visualizaciones como histogramas o diagramas de caja para entender su distribución.
2. **Tratamiento de Datos Nulos**: Según las características observadas, determinaremos el método más adecuado para tratar los datos n


In [ ]:
# Asumimos que df_train_raw['Age'] contiene la edad y algunos datos nulos. 

mean_age = df_train_raw['Age'].mean()
median_age = df_train_raw['Age'].median()
mode_age = df_train_raw['Age'].mode()[0]  # mode() devuelve una Serie, obtenemos el primer valor en caso de datos multimodales
std_age = df_train_raw['Age'].std()

# Graficando el histograma y la curva de distribución normal
plt.figure(figsize=(10, 6))

# Graficar la distribución de edades usando un histograma
sns.histplot(df_train_raw['Age'].dropna(), bins=30, kde=False, color='blue', stat='density')

# Graficar la curva de distribución normal
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mean_age, std_age)
plt.plot(x, p, 'k', linewidth=2)

# Trazar líneas verticales para la media, mediana y moda de las edades
plt.axvline(mean_age, color='red', linestyle='dashed', linewidth=2)
plt.axvline(median_age, color='green', linestyle='dashed', linewidth=2)
plt.axvline(mode_age, color='orange', linestyle='dashed', linewidth=2)

# Agregar texto para las edades media, mediana y moda
plt.text(mean_age + 1, plt.ylim()[1] / 2, f'Media: {mean_age:.2f}', color='red')
plt.text(median_age + 1, plt.ylim()[1] / 2, f'Mediana: {median_age:.2f}', color='green')
plt.text(mode_age - 5, plt.ylim()[1] / 2, f'Moda: {mode_age:.2f}', color='orange')

# Título y etiquetas
plt.title('Distribución de la Edad vs. Distribución Normal')
plt.xlabel('Edad')
plt.ylabel('Densidad')

# Mostrar el gráfico
plt.show()


### Rellenando Valores Nulos en la Columna de Edad con la Mediana

Usar la mediana de la columna 'Edad' para llenar valores nulos está bien justificado por varias razones:

1. **Robustez a Valores Atípicos**: La mediana es menos sensible a valores extremos en los datos. En un conjunto de datos donde la edad puede variar significativamente, algunas edades muy altas o muy bajas pueden sesgar la media, pero no tendrán el mismo efecto en la mediana. Esto hace que la mediana sea una medida más confiable para la tendencia central en este contexto.

2. **Manejo de Distribución Sesgada**: Si la distribución de edades en tu conjunto de datos no es simétrica (es decir, está sesgada), la mediana proporciona una mejor representación de la tendencia central que la media. La media puede ser engañosa en distribuciones sesgadas, ya que se ve arrastrada hacia la cola.

3. **Practicidad y Representatividad**: La mediana, al ser un valor real del conjunto de datos, a menudo tiene más sentido práctico. Representa un valor que se observa de manera realista en los datos, lo que puede no ser el caso con la media, especialmente si la media no es un número entero en el contexto de la edad.

4. **Preservación de la Estructura de Datos**: Usar la mediana ayuda a mantener la distribución general de los datos de edad. Esto es particularmente importante en el modelado estadístico, ya que asegura que la integridad y la representatividad del conjunto de datos se mantengan. Al mantener la forma de la distribución constante, la mediana evita introducir cambios artificiales que podrían afectar los resultados de cualquier análisis o modelos construidos sobre el conjunto de datos.

En resumen, llenar los valores faltantes de edad con la mediana es una estrategia sólida, especialmente en casos donde se trata con valores atípicos, distribuciones sesgadas o se necesita mantener la integridad del conjunto de datos para un modelado preciso.


In [ ]:
df_train_raw['Age'].fillna(df_train_raw['Age'].median(), inplace=True) # Rellenar los valores faltantes en la columna 'Age' con su mediana".

In [ ]:
# Verificando que la columna 'Age' no tenga valores nulos.
df_train_raw.isna().sum() 

### Columnas HomePlanet, CryoSleep, Cabin, Destination

Para las columnas `HomePlanet`, `CryoSleep`, `Cabin` y `Destination`, optaremos por eliminar las filas que contengan datos nulos. A diferencia de los datos numéricos como la edad, estas columnas contienen datos categóricos u objetos, donde la imputación de valores medios o medianas no es aplicable o no tiene sentido. Por lo tanto, la eliminación de filas con valores faltantes es una opción más adecuada para mantener la integridad y la precisión del análisis.

1. **Evaluación de Impacto**: Antes de proceder con la eliminación, es crucial evaluar cuántas filas se verán afectadas. Esto implica visualizar la cantidad de datos nulos en cada una de estas columnas.
2. **Decisión Basada en Datos**: Una vez comprendida la magnitud de los datos nulos, podremos tomar una decisión informada sobre si la eliminación de estas filas es viable sin comprometer significativamente la calidad y la cantidad del dataset.
3. **Proceso de Eliminación**: Si la cantidad de filas a eliminar es razonable y no afecta de manera crítica el análisis, procederemos con la eliminación de estas filas.
4. **Verificación Post-Eliminación**: Tras la eliminación, es importante realizar una verificación final para asegurarnos de que el proceso se ha completado correctamente y de que el dataset resultante sigue siendo adecuado para el análisis subsiguiente.

Este proceso asegura que manejamos los datos nulos de una manera que respete la naturaleza de las variables categóricas y mantenga la calidad del análisis.


In [ ]:
# Mostramos la cantidad de filas con datos nulos.
df_train_raw.isnull().any(axis=1).sum()

Tras analizar las columnas `HomePlanet`, `CryoSleep`, `Cabin` y `Destination`, hemos identificado un total de 773 filas con datos nulos. Considerando que el tamaño total del dataset es de 8693 filas, estos datos nulos representan aproximadamente el 8.9% del dataset.

Dado que la eliminación de estos datos nulos afectará solo una pequeña fracción del dataset, hemos determinado que es viable proceder con su eliminación. Este porcentaje relativamente bajo sugiere que aún tendremos suficientes datos para entrenar un modelo de Machine Learning de manera efectiva. La eliminación de estas filas nos permitirá mantener la integridad y la relevancia de nuestro análisis sin comprometer significativamente la calidad del dataset.

In [ ]:
# Elimina todas las filas que tengan al menos un dato nulo.
df_train_NoNulls = df_train_raw.dropna()

# Verificando que el nuevo dataset no contenga valores nulos
df_train_NoNulls.isna().sum() 



In [ ]:
# Mostrando la nueva extensión del dataset de entrenamiento

print(len(df_train_NoNulls))

Una vez que hemos eliminado las filas con datos nulos, el siguiente paso es revisar nuevamente los tipos de datos en nuestro dataset. El objetivo de esta revisión es asegurarnos de que los tipos de datos sean los más adecuados para el análisis que vamos a realizar y, si es necesario, modificar la estructura de los datos para optimizarla.

Específicamente, buscaremos transformar ciertas columnas para que contengan valores booleanos y numéricos. Esto se debe a que muchos algoritmos de Machine Learning funcionan mejor con datos numéricos y pueden interpretar más fácilmente los valores booleanos. Por ejemplo:

- Convertir variables categóricas en valores booleanos (True/False) cuando sea posible. Esto es particularmente útil para columnas que contienen dos categorías claramente definidas.
- Transformar variables categóricas con múltiples categorías en variables numéricas mediante técnicas como la codificación one-hot, para representarlas de manera más efectiva en modelos numéricos.
- Asegurarnos de que todas las columnas numéricas tengan el tipo de dato correcto (por ejemplo, `int` o `float`), para facilitar su procesamiento en los modelos de Machine Learning.

Esta etapa es crucial para preparar nuestros datos para el modelado, asegurando que estén en un formato que permita la máxima eficiencia y efectividad en el entrenamiento de modelos.


In [ ]:
# Tipos de datos
data_type = df_train_NoNulls.dtypes
unique_values = df_train_NoNulls.nunique()
print(data_type, "\n")

print(unique_values)

### Análisis de la Columna `Cabin`

La columna `Cabin` es una de las más complejas dentro de nuestro dataset debido a su estructura única. Esta columna sigue el formato `deck/num/side`, donde `side` puede ser "P" (Port) o "S" (Starboard). Para simplificar el análisis y preparar los datos para el modelado, transformaremos esta parte de la columna en valores numéricos.

Procederemos de la siguiente manera:

1. **Separación de las Cabinas**: Primero, separamos la información de la columna `Cabin` en sus componentes individuales: `deck`, `num` y `side`.
2. **Transformación de `side`**: Luego, convertiremos la parte `side` de la columna en valores numéricos. Asignaremos el valor `0` a las cabinas que terminen en "P" (Port) y el valor `1` a las que terminen en "S" (Starboard). Esta conversión nos permitirá utilizar esta información de manera más efectiva en modelos de Machine Learning.
3. **Revisión y Verificación**: Tras la transformación, revisaremos los datos para asegurarnos de que la conversión se haya realizado correctamente y que los valores numéricos reflejen con precisión la información original de la columna `Cabin`.

Este proceso de transformación es esencial para hacer que la información contenida en la columna `Cabin` sea más accesible y utilizable en nuestros análisis y modelos predictivos futuros.



In [ ]:
df_train_NoNulls.head() # Estructura Actual

In [ ]:
# Utilizar .loc para transformar la columna 'Cabin' y manejar valores que no son cadenas
df_train_NoNulls.loc[:, 'Cabin'] = df_train_NoNulls['Cabin'].apply(
    lambda x: 
        0 if isinstance(x, str) and x.endswith('P') else (  # Si x es una cadena y termina en 'P', devuelve 0
        1 if isinstance(x, str) and x.endswith('S') else  # Si x es una cadena y termina en 'S', devuelve 1
        x  # Si x no es una cadena, deja el valor como está
    )
)


In [ ]:
df_train_NoNulls.head() # Estructura 0 y 1 para cabin


### Transformación de las Columnas `HomePlanet` y `Destination`

Para las columnas `HomePlanet` y `Destination`, que contienen una cantidad limitada de valores únicos, optaremos por realizar un mapeo numérico en lugar de convertirlas en valores booleanos con la técnica de `get_dummies`. Aunque `get_dummies` es útil para transformar datos categóricos en un formato adecuado para modelos de Machine Learning, en este caso, preferimos evitar la creación de columnas adicionales para mantener la simplicidad y manejo del dataset.

El proceso para cada columna será el siguiente:

1. **Mapeo Numérico de `HomePlanet`**: Asignaremos valores numéricos a cada uno de los planetas de origen únicos. Por ejemplo, podríamos asignar `0` a `Earth`, `1` a `Mars`, y `2` a `Europa`.
2. **Mapeo Numérico de `Destination`**: De manera similar, asignaremos valores numéricos a cada destino único. Cada destino recibirá un número que lo represente.
3. **Revisión y Verificación**: Después de realizar estos mapeos, revisaremos los datos para asegurarnos de que las transformaciones reflejen con precisión la información original de cada columna y que los nuevos valores numéricos sean coherentes y útiles para el análisis.

Esta estrategia nos permite mantener un equilibrio entre la transformación eficiente de variables categóricas y la conservación de un dataset manejable y no excesivamente fragmentado.


In [ ]:
valores_unicos_HomePlanet = df_train_NoNulls['HomePlanet'].unique()
valores_unicos_Destination = df_train_NoNulls['Destination'].unique()

print(valores_unicos_HomePlanet)
print(valores_unicos_Destination)

In [ ]:
# Crear el mapeo de los nombres de los planetas a números
mapeo_planetas = {'Mars': 2, 'Earth': 1, 'Europa': 0}

# Eliminar espacios en blanco alrededor de los valores en la columna 'HomePlanet'
df_train_NoNulls.loc[:, 'HomePlanet'] = df_train_NoNulls['HomePlanet'].str.strip()

# Aplicar el mapeo a la columna 'HomePlanet' en el DataFrame original usando .loc
df_train_NoNulls.loc[:, 'HomePlanet'] = df_train_NoNulls['HomePlanet'].replace(mapeo_planetas)

# Crear el mapeo de los nombres de los destinos a números
mapeo_destinos = {'TRAPPIST-1e': 2, 'PSO J318.5-22': 1, '55 Cancri e': 0}

# Eliminar espacios en blanco alrededor de los valores en la columna 'Destination'
df_train_NoNulls.loc[:, 'Destination'] = df_train_NoNulls['Destination'].str.strip()

# Aplicar el mapeo a la columna 'HomePlanet' en el DataFrame original usando .loc
df_train_NoNulls.loc[:, 'Destination'] = df_train_NoNulls['Destination'].replace(mapeo_destinos)

In [ ]:
df_train_NoNulls.head()


### Conversión de la Columna `PassengerId` a Valores Enteros

La columna `PassengerId` presenta un formato único `gggg_pp`, donde `gggg` representa el grupo al que pertenece el pasajero y `pp` es su número dentro del grupo. Aunque este formato proporciona información valiosa sobre la agrupación de los pasajeros, para fines de análisis y modelado de Machine Learning, es preferible convertir estos identificadores a valores numéricos. Esta conversión permitirá que los algoritmos procesen de manera más eficiente la información relacionada con los grupos de pasajeros.

El proceso de conversión será el siguiente:

1. **Separación de Grupo y Número**: Primero, separaremos el `PassengerId` en dos componentes: el número del grupo (`gggg`) y el número del pasajero dentro del grupo (`pp`). 
2. **Conversión a Valores Numéricos**: Luego, convertiremos ambos componentes en valores enteros. Esto puede implicar tratar el número del grupo como un valor entero más significativo y el número del pasajero como un valor menos significativo, o combinarlos de alguna manera que preserve la relación entre ellos.
3. **Creación de una Nueva Columna**: Podemos crear una nueva columna, como `PassengerGroupID`, que represente esta información numérica, permitiendo así una interpretación más clara de la relación entre los pasajeros y sus grupos.
4. **Revisión y Verificación**: Tras la conversión, es importante revisar los nuevos valores numéricos para asegurarnos de que reflejan adecuadamente la relación original `gggg_pp` y que son útiles para el análisis.

Este enfoque nos permitirá analizar las relaciones entre los pasajeros y sus grupos de una manera más cuantitativa y sistemática, facilitando el descubrimiento de patrones y correlaciones en el dataset.



In [ ]:
# Crear una copia explícita del DataFrame para evitar la advertencia 'SettingWithCopyWarning'
df_train_NoNulls_copy = df_train_NoNulls.copy()

# Realizar las operaciones en la copia
split_columns = df_train_NoNulls_copy['PassengerId'].str.split('_', expand=True)
df_train_NoNulls_copy.loc[:, 'PassengerGroupID'] = split_columns[0].astype(int)
df_train_NoNulls_copy.loc[:, 'PassengerNumber'] = split_columns[1].astype(int)

# Verificar los cambios
print(df_train_NoNulls_copy[['PassengerId', 'PassengerGroupID', 'PassengerNumber']])


In [ ]:
df_train = df_train_NoNulls_copy.copy()
df_train = df_train.drop( columns = "PassengerId")

df_train.head()

In [ ]:
# Reposicionar las ultimas 2 columnas al inicio del dataset

df_train = df_train[df_train.columns[-2:].tolist() + df_train.columns[:-2].tolist()]
df_train.head()

In [ ]:
#Utilizamos el grafico heatmap y pasamos los datos a los parametros del grafico, con la finalidad de visualizar las correlaciones entre las variables
ax = sns.heatmap(df_train.corr(), vmin=-1, vmax=1, annot=True, cmap="YlGnBu", linewidths=1);

In [ ]:
# Verificando que no haya valores nulos antes de crear los tensores de entrenamiento

df_train.isnull().sum()

### Creación de Tensores de Entrenamiento

Con nuestro dataset ya limpio y compuesto solamente por valores booleanos y numéricos, el siguiente paso es la creación del tensor de entrenamiento. Este proceso es crucial para preparar los datos para su uso en modelos de Machine Learning. Los pasos a seguir son:

1. **Definición de Tipos de Datos como Flotantes**: Todos los tipos de datos del dataset deben ser convertidos a flotantes. Esta conversión es importante porque la mayoría de los modelos de Machine Learning, en especial las redes neuronales, requieren de alta precisión en los cálculos, algo que los datos en formato flotante pueden ofrecer.

2. **Mezcla de Datos del Tensor**: Mezclar los datos es un paso esencial para evitar sesgos en el modelo. Al realizar una mezcla aleatoria de las filas del dataset, nos aseguramos de que no haya patrones de aprendizaje basados en el orden de los datos, lo que podría afectar la capacidad del modelo para generalizar a nuevos datos.

3. **Visualización de la Estructura del Tensor (Shape)**: Es importante visualizar la estructura del tensor creado. Conocer el shape del tensor es vital para entender cómo se alimentará al modelo de Machine Learning y para asegurar que la estructura de los datos es la adecuada para el proceso de entrenamiento.

Siguiendo estos pasos, garantizamos que nuestro tensor de entrenamiento esté bien preparado y sea apto para ser utilizado en el entrenamiento de modelos de Machine Learning.




In [ ]:
# Definir el tipo de dato del tensor de entrenamiento
df_float = df_train.astype('float32')

# Convertir el dataset de entrenamiento a un tensor de entrenamiento
tensor_data_train = tf.constant(df_float.values)

# Mezcla del tensor de entrenamiento
tensor_data_train = tf.random.shuffle(tensor_data_train)

# Muestra los primeros 10 datos del tensor
print(tensor_data_train[:10])

# Pasos para Preparar el Dataset de Prueba

Necesitamos que el dataset de prueba esté alineado con el de entrenamiento. Aquí les dejo lo que hay que hacer, paso a paso:

1. **Igualar Estructuras**: Asegúrense de que el dataset de prueba tenga la misma estructura que el de entrenamiento. Mismas columnas, mismos tipos de datos.

2. **Tipos de Datos a Flotantes**: Convertir todos los datos a flotantes, igual que hicimos con el dataset de entrenamiento.

3. **Chequeo de Estructura**: Denle un último vistazo y confirmen que todo está igual que en el de entrenamiento - número de columnas, tipos de datos, etc.

4. **Revisión Final**: Antes de usarlo, revisen todo bien. Cualquier error en la estructura puede afectar los resultados del modelo.




## Preparación de los datos de Prueba

In [ ]:
# Tipos de datos
data_type_test = df_test.dtypes
unique_values_test = df_test.nunique()
print(data_type_test, "\n")

print(unique_values_test)

In [ ]:
# Visualización de la cantidad de valores nulos que se tienen en el dataset de prueba
df_test.isnull().sum()

In [ ]:
# Eliminar columnas 
df_test = df_test.drop(columns=["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Name", "VIP"])

# Mostramos el nuevo dataset sin las columnas
df_test.columns

In [ ]:
# Asumimos que df_train_raw['Age'] contiene la edad y algunos datos nulos. 

mean_age_test = df_test['Age'].mean()
median_age_test = df_test['Age'].median()
mode_age_test = df_test['Age'].mode()[0]  # mode() devuelve una Serie, obtenemos el primer valor en caso de datos multimodales
std_age_test = df_test['Age'].std()

# Graficando el histograma y la curva de distribución normal
plt.figure(figsize=(10, 6))

# Graficar la distribución de edades usando un histograma
sns.histplot(df_test['Age'].dropna(), bins=30, kde=False, color='blue', stat='density')

# Graficar la curva de distribución normal
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mean_age_test, std_age_test)
plt.plot(x, p, 'k', linewidth=2)

# Trazar líneas verticales para la media, mediana y moda de las edades
plt.axvline(mean_age_test, color='red', linestyle='dashed', linewidth=2)
plt.axvline(median_age_test, color='green', linestyle='dashed', linewidth=2)
plt.axvline(mode_age_test, color='orange', linestyle='dashed', linewidth=2)

# Agregar texto para las edades media, mediana y moda
plt.text(mean_age_test + 1, plt.ylim()[1] / 2, f'Media: {mean_age_test:.2f}', color='red')
plt.text(median_age_test + 1, plt.ylim()[1] / 2, f'Mediana: {median_age_test:.2f}', color='green')
plt.text(mode_age_test - 5, plt.ylim()[1] / 2, f'Moda: {mode_age_test:.2f}', color='orange')

# Título y etiquetas
plt.title('Distribución de la Edad vs. Distribución Normal')
plt.xlabel('Edad')
plt.ylabel('Densidad')

# Mostrar el gráfico
plt.show()


In [ ]:
df_test['Age'].fillna(df_test['Age'].median(), inplace=True) # Rellenar los valores faltantes en la columna 'Age' con su mediana".

In [ ]:
# Verificando que la columna 'Age' no tenga valores nulos.
df_test.isna().sum() 

In [ ]:
# Mostramos la cantidad de filas con datos nulos.
df_test.isnull().any(axis=1).sum()

In [ ]:
# Utilizar .loc para transformar la columna 'Cabin' y manejar valores que no son cadenas
df_test.loc[:, 'Cabin'] = df_test['Cabin'].apply(
    lambda x: 
        0 if isinstance(x, str) and x.endswith('P') else (  # Si x es una cadena y termina en 'P', devuelve 0
        1 if isinstance(x, str) and x.endswith('S') else  # Si x es una cadena y termina en 'S', devuelve 1
        x  # Si x no es una cadena, deja el valor como está
    )
)

In [ ]:
df_test.head() # Estructura 0 y 1 para cabin

In [ ]:
valores_unicos_HomePlanet_test = df_test['HomePlanet'].unique()
valores_unicos_Destination_test = df_test['Destination'].unique()

print(valores_unicos_HomePlanet_test)
print(valores_unicos_Destination_test)

In [ ]:
# Crear el mapeo de los nombres de los planetas a números
mapeo_planetas_test = {'Mars': 2, 'Earth': 1, 'Europa': 0}

# Eliminar espacios en blanco alrededor de los valores en la columna 'HomePlanet'
df_test.loc[:, 'HomePlanet'] = df_test['HomePlanet'].str.strip()

# Aplicar el mapeo a la columna 'HomePlanet' en el DataFrame original usando .loc
df_test.loc[:, 'HomePlanet'] = df_test['HomePlanet'].replace(mapeo_planetas_test)

# Crear el mapeo de los nombres de los destinos a números
mapeo_destinos_test = {'TRAPPIST-1e': 2, 'PSO J318.5-22': 1, '55 Cancri e': 0}

# Eliminar espacios en blanco alrededor de los valores en la columna 'Destination'
df_test.loc[:, 'Destination'] = df_test['Destination'].str.strip()

# Aplicar el mapeo a la columna 'HomePlanet' en el DataFrame original usando .loc
df_test.loc[:, 'Destination'] = df_test['Destination'].replace(mapeo_destinos_test)

In [ ]:
df_test.head()

In [ ]:
# Crear una copia explícita del DataFrame para evitar la advertencia 'SettingWithCopyWarning'
df_test_copy = df_test.copy()

# Realizar las operaciones en la copia
split_columns_test = df_test_copy['PassengerId'].str.split('_', expand=True)
df_test_copy.loc[:, 'PassengerGroupID'] = split_columns_test[0].astype(int)
df_test_copy.loc[:, 'PassengerNumber'] = split_columns_test[1].astype(int)

# Verificar los cambios
print(df_test_copy[['PassengerId', 'PassengerGroupID', 'PassengerNumber']])

In [ ]:
df_test = df_test_copy.copy()
df_test = df_test.drop( columns = "PassengerId")

df_test.head()

In [ ]:
# Reposicionar las ultimas 2 columnas al inicio del dataset

df_test = df_test[df_test.columns[-2:].tolist() + df_test.columns[:-2].tolist()]
df_test.head()

In [ ]:
#Utilizamos el grafico heatmap y pasamos los datos a los parametros del grafico, con la finalidad de visualizar las correlaciones entre las variables
ax = sns.heatmap(df_test.corr(), vmin=-1, vmax=1, annot=True, cmap="YlGnBu", linewidths=1);

In [ ]:
# Verificar que no haya valores nulos antes de crear los tensores de prueba
df_test.isnull().sum()

#### **Creación de Tensores de Prueba**

In [ ]:
# Definir el tipo de dato del tensor de entrenamiento
df_test_float = df_test.astype('float32')

# Convertir el dataset de entrenamiento a un tensor de entrenamiento
tensor_data_test = tf.constant(df_test_float.values)

# Muestra los primeros 10 valores del tensor
print(tensor_data_test[:10])

## Definición de los Valores x y y

En la preparación de nuestro modelo de Machine Learning, es esencial definir correctamente los conjuntos de datos `x` y `y`. Estos conjuntos representan, respectivamente, las variables independientes (predictores) y la variable dependiente (lo que queremos predecir).

### Valores x
- **Descripción**: Los valores `x` corresponden a todos los datos del dataset excepto la columna de supervivencia.
- **Objetivo**: Estos valores son los predictores que nuestro modelo utilizará para aprender y hacer predicciones. Incluyen todas las características y atributos relevantes que se consideran influyentes para predecir el resultado.

### Valores y
- **Descripción**: Los valores `y` corresponden a los datos de la columna de supervivencia.
- **Objetivo**: Esta es la variable objetivo que nuestro modelo intentará predecir. Representa el resultado de supervivencia, que es el foco principal de nuestro análisis.

Al separar el dataset en estos dos conjuntos, podemos entrenar nuestro modelo de Machine Learning de manera efectiva, utilizando los valores `x` para predecir los valores `y`.


In [ ]:
# Para los valores de x seleccionamos todas las columnas del tensor de entrenamiento que corresponden a los predictores, a excepción de la ultima que es la variable dependiente

X = tensor_data_train[:, :-1] # Todos los valores de todas las columnas a excepción de la ultima
y = tensor_data_train[:, -1] # Todos los valores de la ultima columna
print(X, "/n")
print(y)

## Expansión de las Dimensiones de `y`

Actualmente, nuestro tensor `y` tiene una forma de (7920,), lo que indica que es unidimensional, mientras que `x` tiene una forma de (7920,7), siendo bidimensional. Es importante que todos los tensores utilizados en el modelado de Machine Learning tengan dimensiones consistentes. Por lo tanto, necesitamos transformar `y` para que también sea un tensor bidimensional. 

### Importancia de la Consistencia de Dimensiones en Machine Learning

- **Compatibilidad**: Algunos frameworks y bibliotecas de Machine Learning requieren que tanto las características (features) como las etiquetas (labels) tengan la misma cantidad de dimensiones para garantizar la coherencia interna durante el entrenamiento.
  
- **Uniformidad**: Tener todos los datos, incluyendo los objetivos o etiquetas, con el mismo número de dimensiones simplifica el manejo y preprocesamiento de los datos.

- **Requisitos del Modelo**: Ciertos modelos y algoritmos están diseñados para trabajar con entradas bidimensionales, tanto para las características como para los objetivos, incluso si el objetivo es un valor único por instancia.

### Reshape de `y` a 2D

Para cumplir con estos requisitos, es una práctica estándar en Machine Learning realizar un reshape de los arrays objetivo 1D a 2D. Este proceso asegura que nuestro tensor `y` sea compatible con los requisitos del modelo y las expectativas de los frameworks utilizados.


In [ ]:
# Expandir dimensiones de 1D a 2D
y = tf.expand_dims(y, axis = -1)
print(y[:5])

## Normalización de Datos

La normalización de los datos es un paso crucial en la preparación de nuestro dataset para el modelado de Machine Learning. Este proceso implica escalar las características de entrada a un rango estándar, lo que puede mejorar significativamente el rendimiento de los modelos de aprendizaje automático. 

### Importancia de la Normalización de Datos

- **Contribución Equitativa de Características**: Al normalizar los datos, aseguramos que todas las características contribuyan de manera equitativa al resultado. Sin normalización, las características con rangos más amplios podrían influir desproporcionadamente en el modelo, llevando a resultados sesgados.

- **Mejora del Gradiente durante el Entrenamiento**: La normalización también es fundamental para garantizar que los gradientes estén bien escalados durante el entrenamiento. Esto facilita que el algoritmo de optimización (como el descenso del gradiente) converja más rápidamente y de manera más efectiva.

### Proceso de Normalización

El proceso de normalización adaptativa implica ajustar las características de entrada para que tengan una media aproximada de 0 y una desviación estándar de 1. Esto se logra restando el valor medio de cada característica y dividiendo por su desviación estándar. Este enfoque es comúnmente conocido como normalización Z-score o estandarización.

Al normalizar los datos de esta manera, no solo mejoramos el rendimiento del modelo, sino que también hacemos que el proceso de entrenamiento sea más estable y consistente, independientemente de las escalas de las características originales.


In [ ]:
# Crear una instancia de la capa de Normalización
normalizador = Normalization()

# "Adaptar" la capa de Normalización con nuestros datos de entrenamiento.
# Esto permite que la capa aprenda las estadísticas (media y desviación estándar) de los datos.
# En términos simples: le estamos diciendo a la capa "Observa estos datos y aprende cómo suelen ser".
normalizador.adapt(X)

# Normalizar los datos 'X' utilizando la capa de normalización y mostrar los primeros 5 elementos.
# Basándose en lo que ha aprendido, la capa transformará estos datos a una escala común.
datos_normalizados = normalizador(X)
datos_normalizados = normalizador(X)
datos_normalizados[:5]


#### Funciones de Activación y Regularizadores

**Funciones de Activación**

Las funciones de activación son cruciales en las redes neuronales, ya que ayudan a propagar el gradiente y permiten que la red aprenda patrones complejos.

**ReLU (Unidad Lineal Rectificada)**
- Ideal para muchas capas ocultas en redes profundas debido a su simplicidad y capacidad de propagar el gradiente sin cambios.
- Fórmula: \( $f(x) = \max(0, x) $\)
- Referencia: [ReLU y sus Variantes en Redes Neuronales](https://arxiv.org/abs/2012.03355)

**Sigmoide**
- Útil en la capa de salida para clasificación binaria, ya que sus valores están entre 0 y 1.
- Fórmula: \($ f(x) = \frac{1}{1 + e^{-x}} $\)
- Referencia: [Redes Neuronales y Regresión Logística](http://www.hlt.utdallas.edu/~vgogate/ml/2019s/lectures/neural-networks.pdf)

**Tanh**
- Similar al sigmoide pero con un rango entre -1 y 1, a menudo se usa en capas ocultas.
- Fórmula: \( $f(x) = \tanh(x) = \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}$ \)
- Referencia: [Función Tangente Hiperbólica y sus Propiedades](https://people.math.sc.edu/girardi/m142/handouts/10s-tanh.pdf)

**Softmax**
- Comúnmente usada en la capa de salida para clasificación multiclase, ya que proporciona una distribución de probabilidades entre múltiples clases.
- Fórmula: \( $f(x)_i = \frac{e^{x_i}}{\sum_{j} e^{x_j}}$ \)
- Referencia: [Understanding the Softmax Function](https://deepai.org/machine-learning-glossary-and-terms/softmax-layer)


La regularización ayuda a prevenir el sobreajuste en el modelo añadiendo una penalización a los pesos durante el entrenamiento.

**Regularización L1**
- Útil cuando se sospecha que muchas características son irrelevantes o cuando se desea un modelo disperso.
- Fórmula de Penalización: \( $P = \alpha \sum |w| $\)
- Referencia: [Regularización L1](https://developers.google.com/machine-learning/glossary/#L1_regularization)

**Regularización L2**
- Ayuda a manejar la colinealidad y es a menudo preferible cuando todas las características son relevantes.
- Fórmula de Penalización: \($ P = \alpha \sum w^2 $\)
- Referencia: [Regularización L2](https://developers.google.com/machine-learning/glossary/#L2_regularization)

Determinar el número de neuronas y capas ocultas requiere experimentación y validación cruzada. Una red con una o dos capas ocultas puede aprender una amplia variedad de patrones; capas adicionales pueden ser útiles para datos más complejos.

Referencia: [Recomendaciones prácticas para el entrenamiento basado en gradiente de arquitecturas profundas](https://arxiv.org/abs/1206.5533)




In [ ]:
def crear_modelo():
    # Inicializar el modelo
    model = Sequential([
        InputLayer(input_shape=(7,)),  # Ajusta el shape de entrada según tu set de entrenamiento
        normalizador,  # Utiliza la capa de normalización que ya has definido
        Dense(64, activation='relu'),  # Primera capa oculta
        Dropout(0.2),  # Capa de dropout
        Dense(32, activation='relu'),  # Segunda capa oculta
        Dense(16, activation='relu'),  # Tercera capa oculta
        Dense(1, activation='sigmoid')  # Capa de salida
    ])

    return model

## Creación de Conjuntos de Datos para Entrenamiento, Validación y Prueba

En el desarrollo de modelos de Machine Learning, es una práctica estándar dividir el dataset principal en tres conjuntos distintos: de entrenamiento, de validación y de prueba. Cada uno de estos conjuntos tiene un propósito específico y es crucial para el desarrollo de un modelo robusto y bien ajustado.

### Dataset de Entrenamiento
- **Uso**: Este conjunto de datos se utiliza para entrenar el modelo. A través de este proceso, el modelo aprende a identificar patrones y a hacer predicciones.
- **Importancia**: Es el mayor conjunto de datos y es crucial para que el modelo aprenda de manera efectiva.

### Dataset de Validación
- **Uso**: Se utiliza para validar el rendimiento del modelo en comparación con el conjunto de entrenamiento. Este conjunto ayuda a ajustar los hiperparámetros y evaluar el sobreajuste.
- **Importancia**: Permite hacer ajustes en el modelo durante el entrenamiento, antes de la evaluación final en el conjunto de prueba.

### Dataset de Prueba
- **Uso**: Este conjunto se utiliza para probar las predicciones del modelo después de que ha sido entrenado y validado.
- **Importancia**: Ofrece una evaluación final y objetiva del rendimiento del modelo en datos no vistos previamente, lo que es crucial para determinar la generalización del modelo.

La correcta separación y uso de estos conjuntos de datos es fundamental para desarrollar un modelo de Machine Learning que sea preciso, generalizable y robusto.


In [ ]:
# Separación de set de entrenamiento y prueba (Después se hace la separación en entrenamiento, validación y pruena con el 90% del dataset)
TRAIN_RATIO = 0.9 
TEST_RATIO = 0.1
DATASET_SIZE = len(X)

# Se crean los conjuntos de entrenamiento que equivalen al 80% del conjunto de datos
X2 = X[:int(DATASET_SIZE*TRAIN_RATIO)]
y2 = y[:int(DATASET_SIZE*TRAIN_RATIO)]
print(X2.shape)
print(y2.shape)

# Se crean los conjuntos de prueba que equivalen al 10% del conjunto de datos
X_test = X[int(DATASET_SIZE*(TRAIN_RATIO)):]
y_test = y[int(DATASET_SIZE*(TRAIN_RATIO)):]
print(X_test.shape)
print(y_test.shape)


## K-FOLDS

### Fase de Compilación del Modelo

En la fase de compilación del modelo, establecemos los parámetros clave que guiarán el proceso de entrenamiento. Estos parámetros incluyen el optimizador, la función de pérdida y las métricas de evaluación.

- **Optimizador Adam**: Seleccionamos el optimizador Adam con una tasa de aprendizaje específica, considerada baja. Esta elección es importante porque una tasa de aprendizaje más baja asegura que el modelo avance gradualmente durante el entrenamiento, evitando saltos demasiado grandes que podrían llevar a perder el mínimo óptimo en la función de pérdida.

- **Función de Pérdida de Entropía Cruzada Binaria**: Esta función es adecuada para problemas de clasificación binaria. Funciona comparando las etiquetas predichas del modelo con las etiquetas verdaderas, proporcionando una medida cuantitativa de la pérdida (o error) que el modelo debe minimizar durante el entrenamiento.

- **Métrica de Precisión**: Como métrica de evaluación, utilizamos la precisión. Esto nos permite medir el porcentaje de instancias que el modelo ha clasificado correctamente, proporcionando una visión clara de la efectividad del modelo durante el entrenamiento y la validación.

Estos componentes son esenciales para asegurar que el modelo se entrene de manera eficiente y efectiva, optimizando su rendimiento y garantizando que los resultados sean confiables y precisos.


In [ ]:
# Número de splits para KFold
n_splits = 5

# Inicializar KFold
kf = KFold(n_splits=n_splits, shuffle=True)

In [ ]:
modelos_entrenados = []
historias = []
resultados = []

for train_index, val_index in kf.split(X2):
    # División de los datos para el fold actual
    X_train_fold, X_val_fold = tf.gather(X2, train_index), tf.gather(X2, val_index)
    y_train_fold, y_val_fold = tf.gather(y2, train_index), tf.gather(y2, val_index)

    # Normalizar los datos
    normalizador.adapt(X_train_fold)
    X_train_fold = normalizador(X_train_fold)
    X_val_fold = normalizador(X_val_fold)

    # Crear una nueva instancia del modelo y compilarla
    model = crear_modelo()  
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])
    # Definición del callback de EarlyStopping
    early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=int(50),# Valor de ajuste
    verbose=1,
    restore_best_weights=True
)

    # Entrenar el modelo con EarlyStopping
    history = model.fit(
        X_train_fold, 
        y_train_fold, 
        validation_data=(X_val_fold, y_val_fold), 
        epochs=1000,
        verbose=1,
        callbacks=[early_stopping]
    )
    
    # Guardar el modelo entrenado en la lista
    modelos_entrenados.append(model)
    
    # Guardar la historia de este fold
    historias.append(history)

    # Evaluar el modelo y guardar los resultados
    resultado = model.evaluate(X_val_fold, y_val_fold)
    resultados.append(resultado)

    # Limpieza de memoria si es necesario
    tf.keras.backend.clear_session()


In [ ]:
# Convertir los resultados a un array de NumPy para facilitar el cálculo
resultados_np = np.array(resultados)

# Calcular promedio y desviación estándar para cada métrica
promedios = np.mean(resultados_np, axis=0)
desviaciones_estandar = np.std(resultados_np, axis=0)

# Imprimir los resultados
print("Promedio de las métricas por fold:")
for i, metrica in enumerate(model.metrics_names):
    print(f"{metrica}: {promedios[i]:.4f} ± {desviaciones_estandar[i]:.4f}")



In [ ]:
# Graficar la pérdida para cada fold
for i, history in enumerate(historias):
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Model Loss - Fold {i+1}')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
# Encontrar el índice del mejor modelo (p. ej., el que tiene la menor pérdida de validación)
indice_mejor_modelo = np.argmin([resultado[0] for resultado in resultados])  # Asumiendo que la posición 0 es la pérdida

# Seleccionar el mejor modelo
mejor_modelo = modelos_entrenados[indice_mejor_modelo]


In [ ]:
print(indice_mejor_modelo)

In [ ]:
# Iterar sobre cada historia de entrenamiento y graficar la precisión
for i, history in enumerate(historias):
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Model Accuracy - Fold {i+1}')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')
    plt.show()

In [ ]:
# Evaluar el modelo sobre el  dataset de entrenamiento
# Esto proporcionará el valor de pérdida y la métrica de precisión para el conjunto de prueba
# Indicando que tan bien se comporta el modelo con nuevos datos
test_loss, test_accuracy = mejor_modelo.evaluate(X_test, y_test)

# Imprimir los resultados de prueba
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Evaluar el modelo sobre el dataset de validación
# Similar a la evaluación de prueba, esto proporcionará el valor de pérdida y la métrica de precisión
# para los datos de validación, los cuales pueden ser monitoreados para el overfitting durante el entrenamiento.
val_loss, val_accuracy = mejor_modelo.evaluate(X_val_fold, y_val_fold)

# Imprimir los resultados de validación
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

In [ ]:
# Extraer los valores verdaderos (etiquetas reales) de los datos de prueba.
# Convertir el tensor a arreglos numpy y posteriormente a listas para una manipulación mas eficiente. 
y_true = list(y_test[:, 0].numpy())

# Usar el modelo para realizar predicciones sobre los datos de prueba
# Después, extraer la primera columna de predicciones (asumiendo que es la columna de interes).
# Convertir el tensor de predicción a una lista
y_pred = list(mejor_modelo.predict(X_test)[:, 0])

# Imprimir las predicciones hechas por el modelo
print(y_pred)

In [ ]:

# Convierte y_pred y y_true a arrays de NumPy si aún no lo son
y_pred = np.array(y_pred)
y_true = np.array(y_true)

# Asegurar que y_pred y y_true tengan la misma longitud
assert len(y_pred) == len(y_true), "Length mismatch between y_pred and y_true."

# Calcula el tamaño de la muestra como el 10% del total
sample_size = int(len(y_pred) * 0.1)

# Selecciona índices aleatorios para representar el 10% de los datos
indices = np.random.choice(len(y_pred), sample_size, replace=False)

# Filtra los datos según los índices seleccionados
y_pred_sample = y_pred[indices]
y_true_sample = y_true[indices]
ind_sample = np.arange(sample_size)  # Ajusta el array ind para la muestra

# Configura el gráfico
plt.figure(figsize=(20, 10))  # Ajusta el tamaño de la figura según sea necesario
width = 0.4  # Ajusta el ancho de las barras

# Dibuja las barras para los datos filtrados
plt.bar(ind_sample, y_pred_sample, width, label='Predicted Transportation')
plt.bar(ind_sample + width, y_true_sample, width, label='Actual Transportation')

# Etiquetas y leyenda
plt.xlabel('Sample Index')
plt.ylabel('Spacer Titanic Transportation')
plt.title('Comparison of Actual vs Predicted Transportation (10% Sample)')
plt.legend()

# Muestra el gráfico
plt.show()

In [ ]:
# Calcular los puntos de la curva ROC y la puntuación AUC
fpr, tpr, _ = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure(figsize=(10, 5))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Calcular la matriz de confusión
cm = confusion_matrix(y_true, np.round(y_pred))

# Graficar la matriz de confusión
plt.figure(figsize=(5, 5))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Calcular la precisión y la recuperación para diferentes umbrales de decisión
precision, recall, _ = precision_recall_curve(y_true, y_pred)

# Graficar la curva de precisión-recuperación
plt.figure(figsize=(10, 5))
disp = PrecisionRecallDisplay(precision=precision, recall=recall)
disp.plot()
plt.title('Precision-Recall curve')
plt.show()

In [ ]:
tensor_data_test.shape

In [ ]:
print(len(df_test))
print(len(tensor_data_test))

In [ ]:
# Hacer predicciones con el mejor modelo
predictions = mejor_modelo.predict(tensor_data_test).flatten()

# Imprime el número de predicciones para asegurarte de que coincide con el número esperado de instancias de prueba.
print(len(predictions))

In [ ]:
# Hacer predicciones con el mejor modelo
predictions = mejor_modelo.predict(tensor_data_test).flatten()

# Convertir las probabilidades predichas en valores booleanos (True o False)
# Aplicando un umbral: las predicciones >= 0.5 se convierten en True, de lo contrario en False
binary_predictions = [True if pred >= 0.5 else False for pred in predictions]

# Crear una nueva columna combinando PassengerGroupID y PassengerNumber con un '_'
# asegurándose de que PassengerGroupID tenga 4 dígitos y PassengerNumber tenga 2 dígitos
df_test['CombinedID'] = df_test['PassengerGroupID'].astype(str).str.zfill(4) + '_' + df_test['PassengerNumber'].astype(str).str.zfill(2)

# Crear el DataFrame output con el CombinedID y las predicciones
output = pd.DataFrame({'PassengerId': df_test['CombinedID'], 'Transported': binary_predictions})


# Save the DataFrame to a CSV file to be used as a submission file
output.to_csv("submission.csv", index=False)
print("Your submission was successfully saved!")